In [39]:
# creating the classes
from pynput import keyboard as kb

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, room):
        self.rooms[room.name] = room

    def add_door(self, door):
        if door.room1.name in self.rooms and door.room2.name in self.rooms:
            self.doors[door.name] = door
            door.room1.add_door(door)
            door.room2.add_door(door)

class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)

    def add_object(self, obj):
        self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("You found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("There's nothing interesting here.")
        if self.doors:
             print(f"You also find that there are {len(self.doors)} doors:")
             for door in self.doors:
                print(f"- {door.name}")
        else:
             print("There are no doors here.")

class Object:
    def __init__(self, name, description, item=None):
        self.name = name
        self.description = description
        self.item = item

    def interact(self, player):
        if self.item:
            print(f"You found a {self.item.name} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Chest(Object):
    def __init__(self, name, description, item = None, is_locked=True, correct_pin = None):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_pin = correct_pin 
    
    def interact(self, player):
        if self.is_locked:
            print(f"The {self.name} is locked, try to open it.! Pin-code is {self.correct_pin}")
            while True:
                pin = input()
                if pin.lower() == "cancel":
                    print("You decided not to try the PIN code.")
                    return  
                try:
                    if len(pin) != 4 or not pin.isdigit():  # check length and if cod contains digits
                        raise ValueError
                    if int(pin) == self.correct_pin:  # check if the pin is correct
                        self.is_locked = False
                        print(f"You unlocked the {self.name}!")
                        super().interact(player)  # open the chest 
                        return  
                    else:
                        print("Incorrect PIN. Try again or type 'cancel':")
                except ValueError:
                    print("Invalid input. Please enter a 4-digit PIN or type 'cancel':")

        else: 
            print("It looks like someone already opened it. There's nothing interesting here.")
    

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.current_action = None
        self.running = True


    def game_loop(self):
        instructions = "Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit."
        further_instructions = "To move through a door or interact with an object, please press the corresponding number of your choice."

        print(f"\nYou are in {self.current_room.name}. {self.current_room.description}")
        print(instructions + " " + further_instructions)
        print("Use 'H' to display instructions again at any point.")

        def on_press(key):
            try:
                if self.current_action is None:
                    if key.char == "h":  # Show instructions
                        print("\n--- Instructions ---")
                        print(instructions)
                        print("--------------------")
                    elif key.char == "w":
                        self.move()
                    elif key.char == "q":
                        self.search_room()
                    elif key.char == "e":
                        self.interact_with_object()

                elif self.current_action == "moving" and key.char.isdigit():
                    self.handle_move(int(key.char))

                elif self.current_action == "interacting" and key.char.isdigit():
                    self.handle_interaction(int(key.char))

            except AttributeError:
                if key == kb.Key.esc:
                    if self.current_action is None:
                        print("Exiting game...")
                        self.running = False  # Stop the game loop
                        return False  # Stops listener
                    else:
                        print("Exiting action...")
                        self.current_action = None

        listener = kb.Listener(on_press=on_press)
        listener.start()

        while self.running:
            pass

        listener.stop()

        print("Game loop ended.")


    def move(self):
        if self.current_action:
            return
        
        self.current_action = "moving"
        doors = self.current_room.doors
        if not doors:
            print("There are no doors here. How did you even get here?")
            self.current_action = None
            return
        
        print("\nWhere do you want to move?")
        for index, door in enumerate(doors, start=1):
            print(f"{index} - {door.name}")
        print("Press a number to select a door or 'Esc' to cancel.")


    def handle_move(self, index):
        if self.current_action != "moving":
            return
        
        doors = self.current_room.doors
        if 1 <= index <= len(doors):
            selected_door = doors[index - 1]
            other_room = selected_door.get_other_room(self.current_room)
            if selected_door.key_required and selected_door.key_required not in self.inventory:
                print(f"You need a {selected_door.key_required.name} to go there.")
            else:
                self.current_room = other_room
                print(f"You moved to {other_room.name}. {other_room.description}")
                
                if self.current_room.name == "Outside":
                    print("Congratulations! You've escaped the haunted mansion!")
                    self.running = False  # Stop the game loop
                    return False

        self.current_action = None  # Exit moving mode
    

    def search_room(self):
        self.current_room.search()


    def interact_with_object(self):
        if self.current_action:
            return
        
        self.current_action = "interacting"
        objects = self.current_room.objects
        if not objects:
            print("There's nothing to interact with here.")
            self.current_action = None
            return

        print("\nWhat do you want to interact with?")
        for index, obj in enumerate(objects.values(), start=1):
            print(f"{index} - {obj.name}")
        print("Press a number to interact or 'Esc' to cancel.")
        

    def handle_interaction(self, index):
        if self.current_action != "interacting":
            return
        
        objects = self.current_room.objects
        if 1 <= index <= len(objects):
            obj = list(objects.values())[index - 1]
            obj.interact(self)

        self.current_action = None  # Exit interaction mode

In [40]:
# creating & setting the game

# Create a house
house = HauntedMansion()

# Create rooms
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.")
hallway = Room("Hallway", "An empty hallway.")
bedroom = Room("Bedroom", "A peaceful bedroom with a soft bed.")
dining_room = Room("Dining Room", "A nice room to have a meal.")
outside = Room("Outside", "A breeze of fresh air and singing birds.")

silver_key = Item("Silver key", "Opens the door between the dining room and the kitchen")
golden_key = Item("Golden key", "Opens the door between the hallway and the dining room")
master_key = Item("Master key", "Opens the big door")


# Add rooms to the house
house.add_room(kitchen)
house.add_room(hallway)
house.add_room(bedroom)
house.add_room(dining_room)
house.add_room(outside)


silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway)
gold_door = Door("Gold door", hallway, bedroom, golden_key)
big_door = Door("Big door", hallway, outside, master_key)



# Connect rooms with doors
house.add_door(silver_door)
house.add_door(white_door)
house.add_door(gold_door)
house.add_door(big_door)



# Create a player and set their starting room
player = Player()
player.current_room = dining_room

# Create a drawer that contains a key
chest = Chest("Chest", "A mysterious chest.", golden_key, correct_pin = 1920)
drawer = Object("Drawer", "An old wooden drawer.", silver_key)
bed = Object("Bed", "A large bed.", master_key)

# Add the drawer to the kitchen
dining_room.add_object(drawer)
kitchen.add_object(chest)
bedroom.add_object(bed)

In [41]:
player.game_loop()


You are in Dining Room. A nice room to have a meal.
Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit. To move through a door or interact with an object, please press the corresponding number of your choice.
Use 'H' to display instructions again at any point.

Where do you want to move?
1 - Silver door
2 - White door
Press a number to select a door or 'Esc' to cancel.
You need a Silver key to go there.

What do you want to interact with?
1 - Drawer
Press a number to interact or 'Esc' to cancel.
You found a Silver key inside the Drawer!

Where do you want to move?
1 - Silver door
2 - White door
Press a number to select a door or 'Esc' to cancel.
You moved to Kitchen. A warm kitchen with a smell of fresh bread.

What do you want to interact with?
1 - Chest
Press a number to interact or 'Esc' to cancel.
The Chest is locked, try to open it.! Pin-code is 1920
You unlocked the Chest!
You found a Golden key inside the Chest!

Where do you want to move?
1 - Silver door
Press a nu